In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functions import *


# declare the netowrk with input size a
class Net(nn.Module):
    def __init__(self, input_size, output_size, activation = nn.Identity()):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size + 1, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, output_size)
        self.activation = activation

    def forward(self, x, holding):
        x = torch.cat((x, holding), dim = 0)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

In [12]:
df = pd.read_csv('factor_trading_1.csv')
df.drop(['Unnamed: 0', 'index', 'long return', 'short return'], axis=1, inplace=True)
df

,AskPrice1,BidPrice1,AskPrice2,BidPrice2,predict_short,predict_long,R_square_short,R_square_long,predict_long_sign,predict_short_sign
0,769.5,769.0,770.0,768.5,-0.000700,-0.000406,0.919429,0.781062,-1,-1
1,769.5,769.0,770.0,768.5,0.000085,0.000223,0.932403,0.807919,1,1
2,769.5,769.0,770.0,768.5,-0.000452,-0.000486,0.896491,0.685345,-1,-1
3,769.5,769.0,770.0,768.5,-0.000617,-0.000644,0.926509,0.609772,-1,-1
4,769.5,769.0,770.0,768.5,-0.000604,-0.000593,0.941604,0.536002,-1,-1
...,...,...,...,...,...,...,...,...,...,...
72794,790.5,790.0,791.0,789.5,-0.000118,-0.000653,0.902961,0.968615,-1,-1
72795,790.5,790.0,791.0,789.5,-0.000106,-0.000593,0.838570,0.901045,-1,-1
72796,790.5,790.0,791.0,789.5,-0.000112,-0.000627,0.880870,0.953120,-1,-1
72797,790.5,790.0,791.0,789.5,-0.000107,-0.000641,0.919722,0.900165,-1,-1


In [13]:
# this the function for the 
def train_model(policy_network, value_network, train_data, num_iter):
    value_losses = []
    ppo = []

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for iteration in range(num_iter):
        value_loss = train_value_network(value_network, policy_network, train_data, value_optimizer, 1, epochs = 1)
        value_losses += value_loss
        new_policy_network = Net(10, 3, activation = nn.Softmax()).to(device)
        new_policy_network.load_state_dict(policy_network.state_dict(), strict=False)
        new_policy_network = nn.DataParallel(new_policy_network)
        new_policy_optimizer = optim.Adam(new_policy_network.parameters(), lr = 0.05)
        #new_policy_network = torch.compile(new_policy_network)
            
        ppo_train(new_policy_network, policy_network, value_network, new_policy_optimizer, train_data, batch_size = 1, epochs = 1, epsilon = 0.1, gamma = 1)
        policy_network = new_policy_network

        value_losses.append(value_loss)
        print(value_losses[-1])

    return policy_network, value_network



In [14]:
policy_network = Net(10, 3, activation = nn.Softmax())
value_network = Net(10, 1, activation = nn.Identity())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
policy_network.to(device)
value_network.to(device)

#policy_optimizer = optim.Adam(policy_network.parameters(), lr=0.1)
#value_optimizer = optim.Adam(value_network.parameters(), lr=0.1)

policy_network = nn.DataParallel(policy_network)
value_network = nn.DataParallel(value_network)

oldest_policy_network = Net(10, 3, activation = nn.Softmax()).to(device)
oldest_policy_network.load_state_dict(policy_network.state_dict(), strict=False)

#policy_network = torch.compile(policy_network)
#value_network = torch.compile(value_network)

policy_optimizer = optim.Adam(policy_network.parameters(), lr=0.05)
value_optimizer = optim.Adam(value_network.parameters(), lr=0.05)




In [15]:
#set up the initial case
train_data = df.iloc[:2500, :].to_numpy()
train_data = torch.tensor(train_data, dtype=torch.float32).to(device)
test_data = df.iloc[2500:3000, :].to_numpy()
test_data = torch.tensor(test_data, dtype=torch.float32).to(device)
policy_network, value_network = train_model(policy_network, value_network, train_data, 10)

wealths = []

wealth = wealth_dist(1, policy_network, test_data)
wealths.append(wealth[0])


# use the previous 2500 tick to train the model
# use same model to trading for the next 1000 tick
# every 1000 tick, retrain the model for once or twice
for num_step in range(3000, 30000, 500):

    print(num_step)
    
    train_data = df.iloc[num_step - 2500: num_step, :].to_numpy()
    train_data = torch.tensor(train_data, dtype=torch.float32).to(device)

    test_data = df.iloc[num_step: num_step + 500, :].to_numpy()
    test_data = torch.tensor(test_data, dtype=torch.float32).to(device)

    policy_network, value_network = train_model(policy_network, value_network, train_data, 2)
    # generate one trajectory
    wealth = wealth_dist(1, policy_network, test_data)
    wealths.append(wealth[0].item())




c:\Users\specf\OneDrive\Desktop\Futures-Trading\functions.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bid_ask = torch.tensor(bid_ask).float().to(device)
C:\Users\specf\AppData\Local\Temp\ipykernel_26080\2440621862.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation(self.fc3(x))


[0.11963298171758652]
[4135251.75]
[7867.38232421875]
[34526.046875]
[9818.3681640625]
[889.2509765625]
[989.1701049804688]
[1365.6903076171875]
[669.6553955078125]
[30.598798751831055]
3000
[300.8599853515625]
[304.64874267578125]
3500
[248.4222869873047]
[11.154175758361816]
4000
[248.05621337890625]
[237.7973175048828]
4500
[241.5364990234375]
[3.33444881439209]
5000
[238.11788940429688]
[237.3750762939453]
5500
[236.60169982910156]
[236.71429443359375]
6000
[237.12022399902344]
[237.01002502441406]
6500
[234.52084350585938]
[234.92344665527344]
7000
[234.72354125976562]
[332696.21875]
7500
[234.97654724121094]
[236.74069213867188]
8000
[235.028076171875]
[0.6412453651428223]
8500
[235.76889038085938]
[235.91424560546875]
9000
[236.6556396484375]
[0.018674982711672783]
9500
[0.013720010407269001]
[0.03540125861763954]
10000
[236.23699951171875]
[0.00010297279368387535]
10500
[45158428672.0]
[385529446400.0]
11000
[15099582464.0]
[62606180352.0]
11500
[72455389184.0]
[21682167808.0]


In [17]:
wealths = wealths[1:]
sum(wealths)

-288.0

In [19]:
# baseline trading strategy
holding = 0
cash = 0

for i in range(2500, 30000):
    if df.iloc[i:i + 1, -2].to_numpy() > 0 and holding <=0:
        holding += 1
        cash -= df.iloc[i:i + 1, 0].to_numpy()
    elif df.iloc[i:i + 1, -2].to_numpy() < 0 and holding >=0:
        holding -= 1
        cash += df.iloc[i:i + 1, 1].to_numpy()

print(cash)
print(holding)

cash += holding * df.iloc[50000:50001, 2].to_numpy()
print(cash)


[-1603.5]
1
[-825.]
